In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

23/02/04 17:34:19 WARN Utils: Your hostname, MacBook-Pro-de-Harrison.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
23/02/04 17:34:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/04 17:34:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.json("../datalake/twitter_datascience")

In [4]:
df.show()

+--------------------+--------------------+--------------------+------------+
|                data|            includes|                meta|extract_date|
+--------------------+--------------------+--------------------+------------+
|[{159910293197395...|{[{2022-12-03T18:...|{1621659740962017...|  2023-02-04|
|[{179520938, 1621...|{[{2010-08-17T14:...|{1621659269673390...|  2023-02-04|
|[{159910293197395...|{[{2022-12-03T18:...|{1621658904814829...|  2023-02-04|
|[{159910293197395...|{[{2022-12-03T18:...|{1621658691001778...|  2023-02-04|
|[{2727683563, 162...|{[{2014-08-12T22:...|{1621658477385875...|  2023-02-04|
|[{154615982630840...|{[{2022-07-10T15:...|{1621657851486666...|  2023-02-04|
|[{145496934121732...|{[{2021-11-01T00:...|{1621656921311780...|  2023-02-04|
|[{2579600334, 162...|{[{2014-06-21T01:...|{1621656410026119...|  2023-02-04|
|[{1962532332, 162...|{[{2013-10-15T12:...|{1621655689843154...|  2023-02-04|
|[{151192065143448...|{[{2022-04-07T04:...|{1621655060672401...|

In [5]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- impression_count: long (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    

In [6]:
from pyspark.sql import functions as f

In [7]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- impression_count: long (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [8]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{1599102931973951...|
|{190097582, 16216...|
|{1599102931973951...|
|{176696392, 16216...|
|{1599102931973951...|
|{879829435, 16216...|
|{190097582, 16216...|
|{1599102931973951...|
|{1371551211351121...|
|{1599102931973951...|
|{179520938, 16216...|
|{2704581690, 1621...|
|{1599102931973951...|
|{65882276, 162165...|
|{1599102931973951...|
|{1599102931973951...|
|{1371551211351121...|
|{1599102931973951...|
|{236564056, 16216...|
|{156666863, 16216...|
+--------------------+
only showing top 20 rows



In [9]:
df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- impression_count: long (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [10]:
tweet_df = df.select(f.explode("data").alias("tweets"))\
  .select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text")

In [11]:
tweet_df.show(5)

+-------------------+-------------------+--------------------+-------------------+----------------+----------+-----------+-----------+-------------+--------------------+
|          author_id|    conversation_id|          created_at|                 id|impression_count|like_count|quote_count|reply_count|retweet_count|                text|
+-------------------+-------------------+--------------------+-------------------+----------------+----------+-----------+-----------+-------------+--------------------+
|1599102931973951489|1621659740962017280|2023-02-03T23:59:...|1621659740962017280|               0|         0|          0|          0|            0|@IansThewriter1 \...|
|          190097582|1621659738151981056|2023-02-03T23:59:...|1621659738151981056|               0|         0|          0|          0|            1|RT @cuongcz: #ai ...|
|1599102931973951489|1621659665334419458|2023-02-03T23:59:...|1621659665334419458|               0|         0|          0|          0|            0|@I

In [13]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [14]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [15]:
user_df.show(5)

+--------------------+-------------------+--------------------+--------------+
|          created_at|                 id|                name|      username|
+--------------------+-------------------+--------------------+--------------+
|2022-12-03T18:07:...|1599102931973951489|PROFICIENT ESSAY ...|IansThewriter1|
|2010-09-13T02:13:...|          190097582|        jayeshthakur| jayeshmthakur|
|2010-08-10T05:33:...|          176696392|      Hugo Caballero|Hugo_Caballero|
|2012-10-14T10:32:...|          879829435|                ruk.| flowery_lapis|
|2021-03-15T19:58:...|1371551211351121920|       Python Roboto|  PythonRoboto|
+--------------------+-------------------+--------------------+--------------+
only showing top 5 rows



In [16]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
user_df.coalesce(1).write.mode("overwrite").json('output/user')